## Robustness of NeuralHash to Some Standard Image Processing Transformations

In [1]:
from data import IMAGENETTE
import utils

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os

/Users/neddamj/miniforge3/envs/nhash/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
folder_path = '../images/'
images = IMAGENETTE()
x = images.load()
images.save_to_disk(x, folder_path, num_images=100)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-05-29 10:50:01.899522: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[INFO] Images saved


### Apply the image transformations to the Images

In [3]:
def flip_image(folder_path, file_name):
    if 'flipped' in file_name or 'rotated' in file_name:
        return
    # Open the image file
    image_path = os.path.join(folder_path, file_name)
    image = Image.open(image_path)

    # Flip the image horizontally
    flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # Save the flipped image with a new file name
    flipped_image_path = os.path.join(folder_path, f"{file_name.split('.')[0]}_flipped.{file_name.split('.')[1]}")
    flipped_image.save(flipped_image_path)

    # Close the original image
    image.close()
    return os.path.join(folder_path, f"{file_name.split('.')[0]}_flipped.{file_name.split('.')[1]}")

In [4]:
def rotate_image(folder_path, file_name):
    if 'flipped' in file_name or 'rotated' in file_name:
        return
    # Open the image file
    image_path = os.path.join(folder_path, file_name)
    image = Image.open(image_path)

    # Rotate the image by 90 degrees clockwise
    rotated_image = image.transpose(Image.ROTATE_90)

    # Save the rotated image with a new file name
    rotated_image_path = os.path.join(folder_path, f"{file_name.split('.')[0]}_rotated.{file_name.split('.')[1]}")
    rotated_image.save(rotated_image_path)

    # Close the original image
    image.close()
    return os.path.join(folder_path, f"{file_name.split('.')[0]}_rotated.{file_name.split('.')[1]}")

In [8]:
original_paths, flipped_paths, rotated_paths = [], [], []
original_hashes, flipped_hashes, rotated_hashes = [], [], []
for file in sorted(os.listdir(folder_path)):
    if file.endswith(('.jpg', '.jpeg', '.png')):
        original_paths.append(os.path.join(folder_path, file))
        if original_paths[-1] is not None:
            original_hashes.append(utils.compute_hash(original_paths[-1]))
        
        flipped_paths.append(flip_image(folder_path, file))    
        if flipped_paths[-1] is None:
            flipped_paths.pop(-1)
        else:
            flipped_hashes.append(utils.compute_hash(flipped_paths[-1]))
        

        rotated_paths.append(rotate_image(folder_path, file))    
        if rotated_paths[-1] is None:
            rotated_paths.pop(-1)
        else:
            rotated_hashes.append(utils.compute_hash(rotated_paths[-1]))

In [9]:
l2_threshold = 20
hamming_threshold = 40

### Compute the Hamming Distances and L2 Distortion between the Hashes of the Flipped Sets of Images

In [10]:
# Original and flipped
success = []
for orig_path, flipped_path, orig_hash, flipped_hash in zip(original_paths, flipped_paths, original_hashes, flipped_hashes):
    orig_img, flipped_img = utils.load_img(orig_path), utils.load_img(flipped_path)
    hamming_dist, l2_dist = utils.distance(orig_hash, flipped_hash, "hamming"), utils.distance(orig_img, flipped_img, "l2")
    print(f'{orig_path} & {flipped_path}\t Hamming Distance: {hamming_dist/96 * 100:.2f}%\t L2 Dist: {l2_dist:.2f}')
    if hamming_dist >= hamming_threshold and l2_dist < l2_threshold:
        success.append(1)
    else:
        success.append(0)

../images/1.jpeg & ../images/1_flipped.jpeg	 Hamming Distance: 41.67%	 L2 Dist: 98.77
../images/10.jpeg & ../images/10_flipped.jpeg	 Hamming Distance: 38.54%	 L2 Dist: 76.35
../images/100.jpeg & ../images/100_flipped.jpeg	 Hamming Distance: 33.33%	 L2 Dist: 114.85
../images/100_flipped.jpeg & ../images/11_flipped.jpeg	 Hamming Distance: 65.62%	 L2 Dist: 290.13
../images/100_rotated.jpeg & ../images/12_flipped.jpeg	 Hamming Distance: 48.96%	 L2 Dist: 169.89
../images/10_flipped.jpeg & ../images/13_flipped.jpeg	 Hamming Distance: 60.42%	 L2 Dist: 118.40
../images/10_rotated.jpeg & ../images/14_flipped.jpeg	 Hamming Distance: 57.29%	 L2 Dist: 114.67
../images/11.jpeg & ../images/15_flipped.jpeg	 Hamming Distance: 55.21%	 L2 Dist: 247.35
../images/11_flipped.jpeg & ../images/16_flipped.jpeg	 Hamming Distance: 69.79%	 L2 Dist: 183.37
../images/11_rotated.jpeg & ../images/17_flipped.jpeg	 Hamming Distance: 53.12%	 L2 Dist: 181.88
../images/12.jpeg & ../images/18_flipped.jpeg	 Hamming Distanc

In [11]:
flipped_asr = sum(success)/len(success)
print(f'ASR: {100*flipped_asr:.2f}%')

ASR: 0.00%


### Compute the Hamming Distances and L2 Distortion between the Hashes of the Rotated Sets of Images

In [12]:
# Original and rotated
for orig_path, rotated_path, orig_hash, rotated_hash in zip(original_paths, rotated_paths, original_hashes, rotated_hashes):
    orig_img, rotated_img = utils.load_img(orig_path), utils.load_img(rotated_path)
    hamming_dist, l2_dist = utils.distance(orig_hash, rotated_hash, "hamming"), utils.distance(orig_img, rotated_img, "l2")
    print(f'{orig_path} & {rotated_path}\t Hamming Distance: {hamming_dist/96 * 100:.2f}%\t L2 Dist: {l2_dist:.2f}')
    if hamming_dist >= hamming_threshold and l2_dist < l2_threshold:
        success.append(1)
    else:
        success.append(0)

../images/1.jpeg & ../images/1_rotated.jpeg	 Hamming Distance: 41.67%	 L2 Dist: 124.11
../images/10.jpeg & ../images/10_rotated.jpeg	 Hamming Distance: 58.33%	 L2 Dist: 109.04
../images/100.jpeg & ../images/100_rotated.jpeg	 Hamming Distance: 51.04%	 L2 Dist: 141.43
../images/100_flipped.jpeg & ../images/11_rotated.jpeg	 Hamming Distance: 46.88%	 L2 Dist: 285.40
../images/100_rotated.jpeg & ../images/12_rotated.jpeg	 Hamming Distance: 43.75%	 L2 Dist: 184.28
../images/10_flipped.jpeg & ../images/13_rotated.jpeg	 Hamming Distance: 51.04%	 L2 Dist: 134.65
../images/10_rotated.jpeg & ../images/14_rotated.jpeg	 Hamming Distance: 50.00%	 L2 Dist: 121.79
../images/11.jpeg & ../images/15_rotated.jpeg	 Hamming Distance: 51.04%	 L2 Dist: 244.80
../images/11_flipped.jpeg & ../images/16_rotated.jpeg	 Hamming Distance: 54.17%	 L2 Dist: 189.52
../images/11_rotated.jpeg & ../images/17_rotated.jpeg	 Hamming Distance: 54.17%	 L2 Dist: 159.73
../images/12.jpeg & ../images/18_rotated.jpeg	 Hamming Dista

In [13]:
rotated_asr = sum(success)/len(success)
print(f'ASR: {100*rotated_asr:.2f}%')

ASR: 0.00%
